# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [127]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [128]:
# #cities data file
data_file ="output_data/cities.csv"
# #read cities data file
weather_df = pd.read_csv(data_file)
weather_df = weather_df.reset_index()
weather_df


,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,0,kyra,49.5776,111.9753,17.69,52,9,11.45,RU,1634314471
1,1,1,faranah,10.0404,-10.7434,86.47,62,67,3.18,GN,1634314471
2,2,2,egvekinot,66.3167,-179.1667,11.97,77,28,7.23,RU,1634314472
3,3,3,jamestown,42.0970,-79.2353,74.52,88,90,4.61,US,1634314400
4,4,4,kodiak,57.7900,-152.4072,33.31,80,20,6.91,US,1634314473
...,...,...,...,...,...,...,...,...,...,...,...
562,562,562,orikhiv,47.5673,35.7858,54.23,42,100,7.27,UA,1634314853
563,563,563,palimbang,6.2106,124.1894,78.96,85,95,3.38,PH,1634314853
564,564,564,navahrudak,53.6000,25.8333,48.96,76,100,15.43,BY,1634314853
565,565,565,olhao,37.0286,-7.8411,71.49,78,0,8.05,PT,1634314854


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [129]:
locations = weather_df[["Lat","Lng"]]
Humidity = weather_df['Humidity'].astype(float)

In [130]:
#plot heatmap
fig = gmaps.figure()
#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=10, point_radius=1.5)
    
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [131]:
#Narrow down the cities to fit weather conditions.
filtering_weather_df = weather_df

#Filter the weather with 3 conditions 

#Filtering Temp: 72 < Max Temp < 85 
filtering_weather_df = filtering_weather_df.loc[(filtering_weather_df['Max Temp'] < 85)\
                                                & (filtering_weather_df['Max Temp'] > 72)]
#Filtering Cloudiness: Cloudiness = 0
filtering_weather_df = filtering_weather_df.loc[filtering_weather_df['Cloudiness'] == 0]

#Filtering Wind Speed: Wind Speed < 15
filtering_weather_df = filtering_weather_df.loc[filtering_weather_df['Wind Speed'] < 12]

#Drop any rows with null values
filtering_weather_df = filtering_weather_df.dropna()



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [132]:


hotel_df = filtering_weather_df

# Add a "Hotel Name" to DataFrame
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# rewrite params dict
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# for Loop to identify hotels:
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_response = requests.get(base_url, params=params).json()
    
    try:
        print(hotel_response["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
        
    except: 
        print('Can not find hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NaN"
    
hotel_df

Can not find hotel at providencia
Lipar Hotel
Can not find hotel at shinpokh
مهمان سرای مرصاد
Can not find hotel at jam sahib
The Halfway Inn
Parsian Bam Hotel
Juman Hotel
Tulip Riverside Hotel And Restaurant Jehlum
Salalah Gardens Hotel
Hotel Misión San Miguel


,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
34,34,34,providencia,28.7167,-111.5833,76.21,37,0,4.00,MX,1634314486,NaN
45,45,45,chabahar,25.2919,60.6430,81.16,59,0,8.14,IR,1634314491,Lipar Hotel
145,145,145,shinpokh,34.3292,71.1782,74.61,17,0,1.21,PK,1634314558,NaN
211,211,211,do gonbadan,30.3586,50.7981,82.49,20,0,0.00,IR,1634314609,مهمان سرای مرصاد
427,427,427,jam sahib,26.2958,68.6292,77.02,11,0,3.96,PK,1634314756,NaN
429,429,429,guerrero negro,27.9769,-114.0611,72.64,46,0,1.45,MX,1634314757,The Halfway Inn
475,475,475,bam,29.1060,58.3570,75.00,11,0,10.31,IR,1634314556,Parsian Bam Hotel
488,488,488,umm lajj,25.0213,37.2685,84.47,66,0,7.09,SA,1634314794,Juman Hotel
496,496,496,sarai alamgir,32.9012,73.7559,78.78,19,0,4.76,PK,1634314809,Tulip Riverside Hotel And Restaurant Jehlum
511,511,511,salalah,17.0151,54.0924,78.89,89,0,3.44,OM,1634314816,Salalah Gardens Hotel


In [133]:
#drop NaN
hotel_df=hotel_df.loc[hotel_df['Hotel Name'] != 'NaN']
hotel_df


,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,45,45,chabahar,25.2919,60.6430,81.16,59,0,8.14,IR,1634314491,Lipar Hotel
211,211,211,do gonbadan,30.3586,50.7981,82.49,20,0,0.00,IR,1634314609,مهمان سرای مرصاد
429,429,429,guerrero negro,27.9769,-114.0611,72.64,46,0,1.45,MX,1634314757,The Halfway Inn
475,475,475,bam,29.1060,58.3570,75.00,11,0,10.31,IR,1634314556,Parsian Bam Hotel
488,488,488,umm lajj,25.0213,37.2685,84.47,66,0,7.09,SA,1634314794,Juman Hotel
496,496,496,sarai alamgir,32.9012,73.7559,78.78,19,0,4.76,PK,1634314809,Tulip Riverside Hotel And Restaurant Jehlum
511,511,511,salalah,17.0151,54.0924,78.89,89,0,3.44,OM,1634314816,Salalah Gardens Hotel
515,515,515,tomatlan,19.9333,-105.2500,83.71,64,0,1.14,MX,1634314818,Hotel Misión San Miguel


In [134]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [135]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))